## MODEL ARICHTECTURE USED: 
A recurrent neural network (RNN) named a Pyramid Bidirectional Long Short-Term Memory (pBLSTM) was used to implement automatic speech recognition to successfully complete this task.
A pBLSTM is part of a model called "Listen, Attend and Spell (LAS)"¹. The LAS model consists of two components: a listener and a speller. The listener is a pyramidal RNN encoder that accepts filter phonemes as inputs. The speller is an attention-based recurrent network decoder that emits characters as outputs.
The listener, which is a pyramidal RNN, converts low-level speech signals into higher-level features and it is done using a pyramid BLSTM (pBLSTM). In each successive stacked pBLSTM layer, the time resolution is reduced by a factor of 2; this allows the model to handle longer sequences and capture more complex patterns in the data.

## How to run my code: 

In brief, the aims of this work was to teach us how to solve sequence-to-sequence problems using sequence models. I got to learn how to set up GRU/LSTM models on PyTorch, use CNNs as feature extractors, handle sequential data, pad/pack batches of variable length data, train the model using CTC Loss, optimize the model, and implement decoders such as greedy and beam decoders. I got to also explore various architectures and hyperparameters to find the optimal solution and learn how to stage data to efficiently search through the space of solutions. Finally, I learnt how to use objects from the PyTorch framework to build a GRU/LSTM based model and deal with issues of data loading, memory usage, and arithmetic precision to maximize the time efficiency of training and inference.I learned hyperparameter tuning to ensure that the model perform well.

To fulfill the requirement of this project while achieving the high cutt off accuracy, different steps were taken into consideration like hyperparameter tuning.

Hyperparameters used and resulted in the best score are:
Around 200 epochs used, batch_size of 32, learning rate of 3e-3 and others as stated in configs cell

Data loader functions were implemented to help in loading the data both training, validation and testing set.
## Results
Valid_distance locally: 5.244
Test distance on kaggle (public score): 5.58
Test distance on kaggle (private score): 5.80

## code running

To ensure the code are running, Kaggle is in use with GPU runtime, data are downloaded from kaggle, checkpoints are saved in kaggle/working directory.
you run the code cells from the beginning, no cell will be skipped.

To reload checkpoint, you uncomment codes in cell 59 which contain code to load checkpoints and load the path to the checkpoint and run the cell.





# Installs

In [ ]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q


This may take a while

In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q

# Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

In [ ]:
!pip install speechpy

In [ ]:
import speechpy
import glob

# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"---------","key":"--------------"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c automatic-speech-recognition-asr

In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f23-hw3p2  automatic-speech-recognition-asr.zip	ctcdecode
'''
!unzip -q automatic-speech-recognition-asr.zip
!ls

# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
# You might want to play around with the mapping as a sanity check here

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this task, we give you full flexibility to design your data set class.
      #TODO
    def __init__(self,directory_path,partition,transforms,limit=None,train_data=True):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them
        self.mfcc_dir = f"{directory_path}/{partition}*/mfcc/*" #TODO.    
        self.transcript_dir = f"{directory_path}/{partition}*/transcript/*"#TODO

        self.mfcc_files =sorted(glob.glob(self.mfcc_dir)) #TODO
        self.transcript_files= sorted(glob.glob(self.transcript_dir)) #TODO


        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files) #NotImplemented


        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        self.mfccs, self.transcripts = [], []

        for file in self.mfcc_files:
          self.mfccs.append(np.load(file))
        for file in self.transcript_files:
          temp =np.load(file)[1:-1]
          self.transcripts.append(np.array([self.PHONEMES.index(i) for i in temp]))

        # train data
        if train_data:
          partition_data = 'train-clean-100'
          self.mfcc_files_data = sorted(glob.glob(f"{directory_path}/{partition_data}*/mfcc/*"))
          self.transcript_files_data = sorted(glob.glob(f"{directory_path}/{partition_data}*/transcript/*"))


          for file in self.mfcc_files_data:
            self.mfccs.append(np.load(file))
          for file in self.transcript_files_data:
            temp =np.load(file)[1:-1]
            self.transcripts.append(np.array([self.PHONEMES.index(i) for i in temp]))

        assert len(self.mfccs) == len(self.transcripts)



    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        # raise NotImplemented
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        # raise NotImplemented

        mfcc = torch.FloatTensor(self.mfccs[ind]) # TODO
        transcript = torch.LongTensor(self.transcripts[ind]) # TODO

        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        self.batch = batch
        # batch of input mfcc coefficients
        batch_mfcc = [i[0] for i in self.batch] # TODO
        # batch of output phonemes
        batch_transcript = [i[1] for i in self.batch] # TODO

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = [len(i) for i in batch_mfcc] # TODO

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first = True) # TODO
        lengths_transcript = [len(i) for i  in batch_transcript] # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



### Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):

  def __init__(self,directory_path,partition,transforms,limit=None):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = f"{directory_path}/{partition}*/mfcc/*" #TODO

        self.mfcc_files =sorted(glob.glob(self.mfcc_dir))

        self.PHONEMES = PHONEMES

        self.mfccs = []

        for files in self.mfcc_files:
          self.mfccs.append(np.load(files))

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''


  def __len__(self):
     '''
     TODO: What do we return here?
     '''
     return self.length
     # raise NotImplemented

  def __getitem__(self, ind):
    '''
    TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

    If you didn't do the loading and processing of the data in __init__,
    do that here.

    Once done, return a tuple of features and labels.
    '''
    mfcc = torch.FloatTensor(self.mfccs[ind])

    return mfcc
    # raise NotImplemented


  def collate_fn(self,batch):
      '''
      TODO:
      1.  Extract the features and labels from 'batch'
      2.  We will additionally need to pad both features and labels,
          look at pytorch's docs for pad_sequence
      3.  This is a good place to perform transforms, if you so wish.
          Performing them on batches will speed the process up a bit.
      4.  Return batch of features, labels, lenghts of features,
          and lengths of labels.
      '''
    
      self.batch = batch
        # batch of input mfcc coefficients
      batch_mfcc = self.batch # TODO

      # HINT: CHECK OUT -> pad_sequence (imported above)
      # Also be sure to check the input format (batch_first)
      batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True) # TODO
      lengths_mfcc = [len(i) for i in batch_mfcc] # TODO

      # You may apply some transformation, Time and Frequency masking, here in the collate function;
      # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
      #                  -> Would we apply transformation on the validation set as well?
      #                  -> Is the order of axes / dimensions as expected for the transform functions?

      # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
      return batch_mfcc_pad, torch.tensor(lengths_mfcc)


### Config - Hyperparameters

In [ ]:
root = '/kaggle/working/11-785-f23-hw3p2/'

# Feel free to add more items here
config ={
    'beam_width'        : 5 ,
    'lr'                : 3e-3,
    'epochs'            : 40,
    'batch_size'        : 32,   # Increase if your device can handle it
    'encoder dropout'   : 0.25,
    'decoder dropout'   : 0.15,
    'lstm dropout'      : 0.25,
    'factor'            : 0.5,
    'sc_mode'           : 'min',
    'patience'          : 3,
    'sc_threshold'      : 1e-3,
    'threshold mode'    : 'rel',
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [ ]:
# get me RAMMM!!!!
import gc
gc.collect()

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(directory_path = root, partition='train-clean-100', transforms=[], limit=None,train_data=True) #TODO
val_data = AudioDataset(directory_path= root, partition='dev-clean', transforms=[], limit=None, train_data = False ) # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest(directory_path=root, partition='test-clean', transforms=[], limit=None) #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(dataset = train_data, num_workers=7, batch_size =config['batch_size'],pin_memory=True, shuffle=True, collate_fn= train_data.collate_fn)#TODO
val_loader = torch.utils.data.DataLoader(dataset = val_data, num_workers=4, batch_size =config['batch_size'],pin_memory=True, shuffle=True, collate_fn= val_data.collate_fn)#TODO
test_loader = torch.utils.data.DataLoader(dataset = test_data, num_workers=1, batch_size =config['batch_size'],pin_memory=True, shuffle=False, collate_fn= test_data.collate_fn)#TODO

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
# torch.cuda.empty_cache()

# class Network(nn.Module):

#     def __init__(self):

#         super(Network, self).__init__()

#         # Adding some sort of embedding layer or feature extractor might help performance.
#         # self.embedding = ?

#         # TODO : look up the documentation. You might need to pass some additional parameters.
#         self.lstm = nn.LSTM(input_size = __, hidden_size = 256, num_layers = 1)

#         self.classification = nn.Sequential(
#             #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
#         )


#         self.logSoftmax = #TODO: Apply a log softmax here. Which dimension would apply it on ?

#     def forward(self, x, lx):
#         #TODO
#         # The forward function takes 2 parameter inputs here. Why?
#         # Refer to the handout for hints
#         pass

## Initialize Basic Network
(If trying out the basic Network)

## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = nn.LSTM(input_size=2*input_size,hidden_size=hidden_size,num_layers=1,batch_first=True,bidirectional=True)# TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence
        x_pad, x_seq_len = pad_packed_sequence(x_packed, batch_first= True)

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        x_trunc, x_len = self.trunc_reshape(x_pad, x_seq_len)

        # TODO: Pack Padded Sequence. What output(s) would you get?
        x_packed = pack_padded_sequence(x_trunc, x_len, batch_first= True, enforce_sorted= False)

        # TODO: Pass the sequence through bLSTM
        output, (h_n, c_n) = self.blstm(x_packed)

        # What do you return?

        return output #NotImplemented

    def trunc_reshape(self, x, x_lens):
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)

        x, x_lens = (x[:, :-1, :], x_lens - 1) if x.shape[1] % 2 != 0 else (x, x_lens)

        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        batch_size,in_channels, input_size = x.shape
        # Calculate the new shape for x
        new_channels_shape = in_channels // 2
        new_input_sizeshape = input_size * 2
        # Reshape 'x' using the new dimensions
        x = x.reshape((batch_size,new_channels_shape, new_input_sizeshape ))

        # TODO: Reduce lengths by the same downsampling factor
        x_lens = x_lens//2
        return x, x_lens

In [ ]:
from torch.autograd import Variable

class LockedDropout(nn.Module):
  
  def __init__(self, dropout):
    super().__init__()
    self.dropout = dropout

  def forward(self, x):
    if not self.training or not self.dropout:
      return x
    x_unpacked, x_lens = pad_packed_sequence(x, batch_first= True)
    m = x.data.new(1, x_unpacked.size(1), x_unpacked.size(2)).bernoulli_(1 - self.dropout)
    mask = Variable(m, requires_grad= False) / (1 - self.dropout)
    mask = mask.expand_as(x_unpacked)
    out = mask * x_unpacked
    return pack_padded_sequence(out, lengths= x_lens, batch_first= True, enforce_sorted= False)

### Encoder

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()


        self.embedding = torch.nn.Sequential( PermuteBlock(), 
                                             torch.nn.Conv1d(in_channels= 28, out_channels= 28, kernel_size= 7, padding= 3), 
                                             torch.nn.GELU(),PermuteBlock())
        #TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.
        self.BLSTMs = torch.nn.LSTM(input_size=28, hidden_size= encoder_hidden_size, num_layers=3, 
                                    batch_first= True, bidirectional= True, dropout= config['lstm dropout'])

        self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            # ...
            # ...
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size), LockedDropout(config['encoder dropout']),
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size), LockedDropout(config['encoder dropout']),
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size), LockedDropout(config['encoder dropout'])
        )

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        x_out = self.embedding(x)
        # TODO: Pack Padded Sequence
        x_pad_packed = pack_padded_sequence(x_out, x_lens, batch_first= True, enforce_sorted= False)
        x_blstm, (_,_) = self.BLSTMs(x_pad_packed)
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        x_pblstm = self.pBLSTMs(x_blstm)
        # TODO: Pad Packed Sequence
        encoder_outputs, encoder_lens = pad_packed_sequence(x_pblstm, batch_first= True)
        # Remember the number of output(s) each function returns

        return encoder_outputs, encoder_lens

### Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), 
            torch.nn.BatchNorm1d(2*embed_size), 
            PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
            torch.nn.Linear(2 * embed_size, 2 * embed_size), torch.nn.GELU(),
            torch.nn.Dropout(config['decoder dropout']),
            torch.nn.Linear(2 * embed_size, 2 * embed_size), torch.nn.GELU(),
            torch.nn.Dropout(config['decoder dropout']),
            torch.nn.Linear(2 * embed_size, output_size))


        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        mlp_out = self.mlp(encoder_out)
        #TODO Think what should be the final output of the decoder for the classification
        out = self.softmax(mlp_out)
        
        return out

In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(

            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
        )


        self.encoder        = Encoder(input_size, embed_size)# TODO: Initialize Encoder
        self.decoder        = Decoder(embed_size, output_size)# TODO: Initialize Decoder



    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [ ]:
model = ASRModel(
    input_size  = 28, #TODO
    embed_size  = 256,#TODO
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

In [ ]:
# checkpoint = torch.load('....') # path to checkpoint
# model.load_state_dict(checkpoint['model_state_dict'])

# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [ ]:
#TODO

criterion = nn.CTCLoss(blank= CMUdict.index(''))# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

# optimizer =  torch.optim.AdamW(...) # What goes in here?
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(labels=LABELS, beam_width=config['beam_width'], num_processes= 4, log_probs_input=True)#TODO

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= config['sc_mode'], factor= config['factor'], patience= config['patience'], 
                                                       threshold= config['sc_threshold'], threshold_mode= config['threshold mode'],  verbose=True)#TODO

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    output = torch.transpose(output, 0, 1)
    output, _, _, out_seq_len = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings = []
    
    print(output_lens.shape[0], len(output_lens))

    for i in range(output_lens.shape[0]):
    # TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        logits = output[i][0][:out_seq_len[i][0]]
        try:
            pred_strings.append(''.join(PHONEME_MAP[i] for i in logits))
        except Exception as e:
            print(e)
            print("logits:", logits)
            return
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i] #TODO
        label_string = ''.join(PHONEME_MAP[l] for l in label[i][:label_lens[i]])#TODO
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    # raise NotImplemented
    return dist

# Test Implementation

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

# WandB

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="---------------") #API Key is in your wandb account, under settings (wandb.ai/settings)

In [ ]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    # reinit = True, ### Allows reinitalizing runs when you re-run this cell
    id ='lexrsicc', ### Insert specific run id here if you want to resume a previous run
    resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

# Train Functions

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(h, y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss,val_dist

## Training Setup

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions
last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "/kaggle/working/epochmodelcheck.pth"#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = "/kaggle/working/best_checkpoint.pth"#TODO set best model path


In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr']) #TODO

    train_loss              = train_model(model, train_loader, criterion, optimizer)#TODO
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)#TODO
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()

# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 25 #TODO

test_decoder    = CTCBeamDecoder(labels=LABELS, beam_width=TEST_BEAM_WIDTH, log_probs_input=True) #TODO
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)
        h = torch.permute(h, (1, 0, 2))
    prediction_string= decode_prediction(h, lh, test_decoder, LABELS) # TODO call decode_prediction
    #TODO save the output in results array.
    results.extend(prediction_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()

In [ ]:
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

In [ ]:
!kaggle competitions submit -c automatic-speech-recognition-asr -f submission.csv -m "I made it!"